In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchaudio.transforms as T
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
import math
import librosa
import librosa.display
import torch
import torch.nn as nn
from sklearn.model_selection import KFold
import sklearn
import os
from torchsummary import summary

In [3]:
Data_inlab=pd.read_csv('InLab_NN.csv')
Data_inlab['ibif_acflow']=Data_inlab['ibif_acflow']/1000
Data_inlab['Ps']=Data_inlab['Ps']*98.0665
Data_inlab

,Unnamed: 0,SubjectID,condition,gender,peak_index,pitch,quality,vowel,Ps_m1,Ps_m2,...,ibif_acflow,ibif_mfdr,ibif_h1h2,ibif_oq,ibif_sq,ibif_naq,ibif_hrf,ibif_f0,acc_f0,Ps_NN
0,0,NF002,normal,F,3,comfortable,modal,a,9.260646,11.601520,...,0.161813,671.233943,5.765352,52.233677,486.342593,0.049962,-0.507651,207.253886,204.876040,7.971257
1,1,NF002,normal,F,4,comfortable,modal,a,7.545824,9.634000,...,0.100727,505.531885,8.815717,42.396313,131.085492,0.049635,-5.002770,249.110320,211.877969,8.648448
2,2,NF002,normal,F,5,comfortable,modal,a,5.038262,6.257666,...,0.083088,284.150763,10.068140,68.421053,58.476980,0.061451,-5.164230,210.157618,208.423697,7.278603
3,3,NF002,normal,F,6,comfortable,modal,a,4.076063,5.134156,...,0.047759,184.575708,12.515432,58.496241,79.597354,0.054762,-8.642713,211.640212,210.119605,6.593817
4,4,NF002,normal,F,7,comfortable,modal,a,3.346026,3.478173,...,0.039989,100.684732,16.270406,73.708207,253.476345,0.084280,-13.457584,212.201592,209.700106,5.090527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15155,212,PM057,uvfp,M,2,high,modal,u,8.803225,8.074281,...,0.078829,94.456016,24.569979,70.068027,193.888889,0.171192,-24.427688,205.128205,229.699678,6.085783
15156,213,PM057,uvfp,M,3,high,modal,u,9.005508,9.440667,...,0.056298,76.805754,17.393843,66.052227,116.908415,0.157634,-17.078416,215.053763,245.180074,6.868375
15157,214,PM057,uvfp,M,4,high,modal,u,7.781924,7.389574,...,0.044268,34.167549,20.203928,95.286195,151.626303,0.271331,-19.963130,209.424084,214.208332,5.379981
15158,215,PM057,uvfp,M,5,high,modal,u,8.958859,9.381394,...,0.060002,77.232859,14.256172,51.177394,89.607813,0.171691,-13.872684,220.994475,229.476744,7.436692


In [ ]:
Data_inlab_normal=Data_inlab[Data_inlab['condition']=='normal']
print(np.unique(Data_inlab_normal['SubjectID']))
ID_Subject=Data_inlab_normal['SubjectID'].to_numpy()
Subject=np.unique(ID_Subject)
Data_inlab_normal= pd.DataFrame(Data_inlab_normal, columns=['ibif_acflow','ibif_mfdr','ibif_oq','ibif_sq', 'ibif_f0','ibif_h1h2', 'level_dB', 'Ps'])
Data_inlab_normal.columns =['ACFL','MFDR','OQ_mean','sQ_mean', 'f0','H1H2', 'spl', 'Ps']
dataset_Clinic=Data_inlab_normal
dataset_Clinic

['NF002' 'NF006' 'NF008' 'NF015' 'NF017' 'NF019' 'NF021' 'NF024' 'NF027'
 'NF028' 'NF029' 'NF030' 'NF032' 'NF035' 'NF039' 'NF040' 'NF041' 'NF044'
 'NM001' 'NM003' 'NM009' 'NM013' 'NM016' 'NM022' 'NM026' 'NM031']


,ACFL,MFDR,OQ_mean,sQ_mean,f0,H1H2,spl,Ps
0,0.161813,671.233943,52.233677,486.342593,207.253886,5.765352,94.121538,1130.163310
1,0.100727,505.531885,42.396313,131.085492,249.110320,8.815717,88.996536,878.937787
2,0.083088,284.150763,68.421053,58.476980,210.157618,10.068140,83.062238,599.918722
3,0.047759,184.575708,58.496241,79.597354,211.640212,12.515432,77.470923,462.209312
4,0.039989,100.684732,73.708207,253.476345,212.201592,16.270406,72.171007,405.306744
...,...,...,...,...,...,...,...,...
8288,0.060065,92.934768,64.297521,92.655784,165.975104,18.627020,74.763840,451.089454
8289,0.053618,95.646861,61.196581,111.761482,171.673820,17.110855,74.532837,459.574568
8290,0.085968,106.846702,40.161290,65.241265,164.271047,21.148496,72.674284,462.620422
8291,0.072425,114.177928,72.203390,298.426760,169.133192,18.756192,73.886476,422.718791


In [5]:
dataset_Clinic.describe().T

,count,mean,std,min,25%,50%,75%,max
ACFL,8293.0,0.171374,0.192922,0.001077,0.049194,0.109041,0.221522,1.878599
MFDR,8293.0,360.444024,394.375247,2.036439,92.780517,224.085673,482.822537,3165.379937
OQ_mean,8293.0,54.797064,12.153104,4.388715,46.699670,54.844007,62.318841,99.666667
sQ_mean,8293.0,156.729256,88.245790,1.184465,115.305250,142.230156,176.608225,3788.942029
f0,8293.0,238.741917,94.926851,65.789474,170.212766,222.222222,287.769784,628.930818
H1H2,8293.0,13.007991,7.476387,-0.000000,8.029480,12.242365,17.170730,57.482502
spl,8293.0,79.404878,8.410436,46.717462,73.430305,79.647695,85.507213,105.521610
Ps,8293.0,741.661234,355.228875,100.216374,481.407228,680.238322,927.853300,2638.920878


In [6]:
def norm(x):
  return (x - train_stats['min']) / (train_stats['max']- train_stats['min'] )

def norm_labels(x):
  return (x - labels_train_stats['min']) / (labels_train_stats['max']- labels_train_stats['min'] )


In [7]:
class MLP(nn.Module):


        def __init__(self, neurons_MLP=128, drop_out=0.2):
            super().__init__()
            # conv block
            self.MLP_1 = nn.Sequential(
            nn.Linear(7,256),
            nn.ReLU(),
            nn.Dropout(p=drop_out),
            )
            self.MLP_2 = nn.Sequential(
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(p=drop_out),
            )
            self.MLP_3 = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(p=drop_out),
            )
            self.MLP_4 = nn.Sequential(
            nn.Linear(64,1),
            )



        def forward(self,x):
            x1 = self.MLP_1(x)
            x2 = self.MLP_2(x1)
            x3 = self.MLP_3(x2)

            x4 = self.MLP_4(x3)
            return x4

In [ ]:
train_dataset_Clinic = dataset_Clinic

train_stats = train_dataset_Clinic.describe()
labels_train_stats =pd.DataFrame([train_stats.pop(x) for x in ["Ps"]])
train_stats = train_stats.transpose()

train_labels_Clinic = pd.DataFrame([train_dataset_Clinic.pop(x) for x in ["Ps"]]).T


normed_train_data_Clinic = norm(train_dataset_Clinic).to_numpy()

normed_train_labels_Clinic = norm_labels(train_labels_Clinic).to_numpy()



In [9]:
def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device=='cuda':
    torch.cuda.set_device(1)

BATCH_SIZE = 64


sgkf = sklearn.model_selection.LeaveOneGroupOut()

RMSE_Ps, True_Value, NN_prediction =[],[],[]
MAE_Ps, MAPE_Ps, R2_Ps =[],[],[]
for k, (train_index, val_index) in enumerate(sgkf.split(normed_train_data_Clinic, normed_train_labels_Clinic, ID_Subject)):

    print('')
    print(f"\k-fold {k + 1} \n***********************")
    print(np.unique(ID_Subject[val_index]))
    X_train=normed_train_data_Clinic[train_index]
    Y_train= normed_train_labels_Clinic[train_index]
    X_train_tensor = torch.tensor(X_train,device=device).float()
    Y_train_tensor = torch.tensor(Y_train,device=device).float()

    X_test=normed_train_data_Clinic[val_index]
    Y_test= normed_train_labels_Clinic[val_index]
    X_test_tensor = torch.tensor(X_test,device=device).float()
    Y_test_tensor = torch.tensor(Y_test,device=device).float()


    results_loss=[]

    DATASET_SIZE=X_train.shape[0]
    iters = int(DATASET_SIZE / BATCH_SIZE)

    model = MLP().to(device)
    model.apply(reset_weights)

    model.load_state_dict(torch.load(os.path.join('Model_syntethic_6.pt')))
    for param in model.MLP_1.parameters():
    	param.requires_grad = False

    model.MLP_4.apply(reset_weights)
    model.MLP_3.apply(reset_weights)
    model.MLP_2.apply(reset_weights)

    loss_func = nn.MSELoss()  # mean square error
    lambda1 = lambda epoch: 0.95 ** epoch
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
    dataset=TensorDataset(X_train_tensor,Y_train_tensor)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    for epoch in range(100):
        epoch_acc = 0
        epoch_loss = 0
        for id_batch, (x_batch, y_batch) in enumerate(dataloader):
            actual_batch_size=x_batch.shape[0]

            # zero the parameter gradients
            model.train()
            optimizer.zero_grad()

            # forward
            outputs = model(x_batch)


            # calculate loss + backward + optimize
            loss = loss_func(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()*actual_batch_size/DATASET_SIZE
    print(f"\r Epoch {epoch +1}: iteration {id_batch +1}/{iters} RMSE: {np.sqrt(epoch_loss):.2f}",end='')
   
    scheduler.step()
    results_loss.append(epoch_loss)

    with torch.no_grad():
        model.eval()
        output_test = model(X_test_tensor)

    test_predictions=output_test[:,0]*(labels_train_stats['max'][0]- labels_train_stats['min'][0] ) + labels_train_stats['min'][0]
    Real_label=Y_test_tensor[:,0]*(labels_train_stats['max'][0]- labels_train_stats['min'][0] ) + labels_train_stats['min'][0]


    RMSE_Clinic = loss_func(Real_label.data.cpu()/98.0665, test_predictions.data.cpu()/98.0665)
    MAE_Clinic=np.mean(np.abs(Real_label.data.cpu().numpy()-test_predictions.data.cpu().numpy())/98.0665)
    MAPE_Clinic=np.mean(np.abs((Real_label.data.cpu().numpy()-test_predictions.data.cpu().numpy())/Real_label.data.cpu().numpy()))*100
    R2_Clinic=sklearn.metrics.r2_score(Real_label.data.cpu().numpy(), test_predictions.data.cpu().numpy())

    RMSE_Ps.append(np.sqrt(RMSE_Clinic.item()))
    MAE_Ps.append(MAE_Clinic)
    MAPE_Ps.append(MAPE_Clinic)
    R2_Ps.append(R2_Clinic)
    print('')
    print('Error',np.sqrt(RMSE_Clinic.item()))
    if k==0:
      True_Value=Real_label
      NN_prediction=test_predictions
    else:
      True_Value=torch.concat([True_Value,Real_label])
      NN_prediction=torch.concat([NN_prediction,test_predictions])
    


\k-fold 1 
***********************
['NF002']
 Epoch 100: iteration 129/128 RMSE: 0.07
Error 1.6241431544823353

\k-fold 2 
***********************
['NF006']
 Epoch 100: iteration 127/126 RMSE: 0.07
Error 1.1024010895456542

\k-fold 3 
***********************
['NF008']
 Epoch 100: iteration 127/126 RMSE: 0.07
Error 1.1444384957182594

\k-fold 4 
***********************
['NF015']
 Epoch 100: iteration 127/126 RMSE: 0.07
Error 1.8806577516989285

\k-fold 5 
***********************
['NF017']
 Epoch 100: iteration 126/125 RMSE: 0.07
Error 0.9675281571817821

\k-fold 6 
***********************
['NF019']
 Epoch 100: iteration 126/125 RMSE: 0.07
Error 1.6098631331513429

\k-fold 7 
***********************
['NF021']
 Epoch 100: iteration 126/125 RMSE: 0.07
Error 3.6763506943843898

\k-fold 8 
***********************
['NF024']
 Epoch 100: iteration 123/122 RMSE: 0.07
Error 1.4908164551179135

\k-fold 9 
***********************
['NF027']
 Epoch 100: iteration 122/121 RMSE: 0.06
Error 3.303689143

In [11]:
print(f"Ps RMSE: {np.mean(RMSE_Ps)} (  {np.std(RMSE_Ps)}) ")
print(f"Ps MAE: {np.mean(MAE_Ps)} (  {np.std(MAE_Ps)}) ")
print(f"Ps R2: {np.mean(R2_Ps)} (  {np.std(R2_Ps)}) ")
print(f"Ps MAPE: {np.mean(MAPE_Ps)} (  {np.std(MAPE_Ps)}) ")

RMSE_all=np.sqrt(np.mean(np.square(True_Value.cpu().numpy()-NN_prediction.cpu().numpy())))/98.065
MAE=np.mean(np.abs(True_Value.cpu().numpy()-NN_prediction.cpu().numpy())/98.0665)
MAPE=np.mean(np.abs((True_Value.cpu().numpy()-NN_prediction.cpu().numpy())/True_Value.cpu().numpy()))*100
MAPE_Max=np.mean(np.abs((True_Value.cpu().numpy()-NN_prediction.cpu().numpy())/np.max(True_Value.cpu().numpy())))*100
ME=np.mean(True_Value.cpu().numpy()-NN_prediction.cpu().numpy())/98.065
R2=sklearn.metrics.r2_score(True_Value.cpu().numpy(),NN_prediction.cpu().numpy())
print("Metrics Mean (STD): RMSE: {:5.2f} cmH2O, MAE: {:5.2f} cmH2O, MAPE: {:5.2f} %,MAPE_max: {:5.2f} %, ME: {:5.2f} cmH2O R2: {:5.2f} cmH2O ".format(RMSE_all, MAE, MAPE, MAPE_Max,  ME, R2,))


Ps RMSE: 2.0349301490244462 (  0.8057720210220845) 
Ps MAE: 1.5960354804992676 (  0.6096197962760925) 
Ps R2: 0.49776507231907924 (  0.5887039558810806) 
Ps MAPE: 23.163528568469562 (  9.428886116416093) 
Metrics Mean (STD): RMSE:  2.26 cmH2O, MAE:  1.68 cmH2O, MAPE: 24.08 %,MAPE_max:  6.23 %, ME:  0.23 cmH2O R2:  0.61 cmH2O 
